In [1]:
import gensim
import itertools
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from pprint import pprint
import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import jieba
import matplotlib.pyplot as plt 
import pandas as pd
import nltk

In [26]:
dat = pd.read_csv('./text/text.csv')
print(dat.query('rating == "bad"'))

     Unnamed: 0                                     text rating
0             0                      对不起，我实在是无法接受这个结局。\n    bad
1             1  从哈一到哈七。。。水平依次下降。。越写越不如前，作家都免不了这样的杯具么？\n    bad
2             2                      这套书根本就是在宣扬扭曲的价值观吧\n    bad
3             3                  翻译太烂。和上一本一样，还比不上网译版本。\n    bad
4             4                                    糟透了\n    bad
..          ...                                      ...    ...
101         101                               不是我喜欢的路线\n    bad
102         102                                   平衡一下\n    bad
103         103                          教授死了过于伤心Ծ‸Ծ差评\n    bad
104         104                   @2008-11-21 21:38:38\n    bad
105         105                                   差到想吐\n    bad

[106 rows x 3 columns]


In [13]:
dat['text'] = dat.text.apply(lambda x: ",".join(jieba.cut(x)))
tweets = [t.split(',') for t in dat.text]
f = open('./code/ChineseStopWords.txt')
stopwords = f.read().splitlines()
f.close()
stopwords += ['....', '.....', 'end']
for i in range(len(tweets)):
    tweets[i] = [w for w in tweets[i] if w not in stopwords and len(w)>2]
text = dat.text.values.tolist()
words_list = list(itertools.chain(*tweets))

In [14]:
print(tweets[0:5])

[['对不起'], ['从哈一到', '免不了'], ['根本就是', '价值观'], ['比不上'], ['糟透了']]


In [15]:
id2word = Dictionary(tweets)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tweets]
print(corpus[:1])
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

[[(0, 1)]]


[[('对不起', 1)]]

In [16]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=3, 
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"第七部" + 0.020*"中规中矩" + 0.020*"双胞胎" + 0.016*"大结局" + 0.011*"阿不思" + '
  '0.010*"虎头蛇尾" + 0.010*"英文版" + 0.008*"看不下去" + 0.007*"第四部" + 0.007*"回不来"'),
 (1,
  '0.092*"斯内普" + 0.017*"舍不得" + 0.017*"出乎意料" + 0.015*"大难不死" + 0.015*"三人组" + '
  '0.014*"弗雷德" + 0.014*"说实话" + 0.012*"第一本" + 0.011*"马尔福" + 0.010*"2019"'),
 (2,
  '0.115*"哈利波" + 0.042*"越来越" + 0.022*"一口气" + 0.020*"一般般" + 0.014*"英雄主义" + '
  '0.011*"儿童文学" + 0.010*"中文版" + 0.010*"囫囵吞枣" + 0.010*"哈哈哈哈" + 0.010*"哈哈哈"')]


In [17]:
pyLDAvis.enable_notebook()
p = gensimvis.prepare(lda_model, corpus, id2word)
p

/home/competition/anaconda3/envs/r_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.017713 -0.083254       1        1  35.140047
1     -0.088262  0.053015       2        1  33.362437
2      0.105974  0.030239       3        1  31.497515, topic_info=     Term       Freq      Total Category  logprob  loglift
35    哈利波  43.000000  43.000000  Default  30.0000  30.0000
13    斯内普  36.000000  36.000000  Default  29.0000  29.0000
9     越来越  16.000000  16.000000  Default  28.0000  28.0000
108   第七部   9.000000   9.000000  Default  27.0000  27.0000
81    一口气   8.000000   8.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
570  虎头蛇尾   3.529224   9.172475   Topic3  -4.6578   0.2001
691   没想到   1.400215   2.264052   Topic3  -5.5822   0.6747
690   好莱坞   1.400013   2.264096   Topic3  -5.5824   0.6746
706   打游击   2.133697   5.022795   Topic3  -5.1610   0.2991
714  不能平静   2.059646   5.003175   Topic3  -5.1963   0.2677

[156 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
78        1  0.732202  2008
718       1  0.190765  2019
718       2  0.763062  2019
718       3  0.190765  2019
132       2  0.501081  2020
...     ...       ...   ...
80        1  0.988153   阿不思
110       2  0.959301   马尔福
713       1  0.207127   魔法师
713       2  0.207127   魔法师
713       3  0.621382   魔法师

[164 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])